# FEATURE SELECTION

En este notebook vamos a ver las diferentes maneras de seleccionar aquellas features más relevantes con respecto al target. Esto es importante ya que un exceso de features puede hacer el ajuste muy complejo y poco generalizable (overfitting). 

Destacan dos tipos de técnicas de feature selection: 

- Tipo filter, en los cuales los features se filtran según algún criterio y threshold
- Tipo wrapper, en los cuales se usan otros métodos de clasificación para añadir o quitar features

A continuación, vamos a ver cómo las diferentes tipos están implementados en scikit

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

In [ ]:
from sklearn.datasets import load_breast_cancer

In [ ]:
data = load_breast_cancer()

In [ ]:
X= data['data']
y= data['target']

In [ ]:
X.shape

In [ ]:
X_df = pd.DataFrame(X)
X_df.columns = data['feature_names']

data_df = pd.concat([X_df, pd.DataFrame(y,columns=['benign'])], axis=1)

In [ ]:
data_df.shape

In [ ]:
fig, axs = plt.subplots(ncols=5, nrows=6, squeeze=False, sharex=True, figsize=(15,15))

axs = axs.flatten()
for i,col in enumerate(data['feature_names']):
    pd.DataFrame.boxplot(data_df, column=col, by = 'benign', ax=axs[i])

In [ ]:
#https://matplotlib.org/gallery/images_contours_and_fields/image_annotated_heatmap.html
from mpl_toolkits.axes_grid1 import make_axes_locatable
feat_cors = data_df.loc[:,data['feature_names']].corr().values

fig, ax = plt.subplots(figsize=(15,15))
im = ax.imshow(feat_cors)

ax.set_xticks(np.arange(len(data['feature_names'])))
ax.set_yticks(np.arange(len(data['feature_names'])))


ax.set_xticklabels(data['feature_names'])
ax.set_yticklabels(data['feature_names'])

plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

for i in range(len(data['feature_names'])):
    for j in range(len(data['feature_names'])):
        text = ax.text(j, i, np.round(feat_cors[i,j],1),
                       ha="center", va="center", color="w")
fig.tight_layout()
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)
plt.show()

Veamos primero cómo es la clasificación usando un árbol de decisión usando todas las features

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

skf = StratifiedKFold(n_splits=10, random_state=0)
clf = DecisionTreeClassifier()
print(np.mean(cross_val_score(clf, X, y, cv=skf)))

Parece que hay muchas correlaciones. Vamos a eliminar aquéllas más correlacionadas y ver cómo cambia la clasificación

In [ ]:
to_drop = ['mean perimeter','mean radius','mean compactness','mean concave points',
              'radius error','perimeter error','worst radius','worst perimeter',
              'worst compactness','worst concave points','compactness error',
              'concave points error','worst texture','worst area']

In [ ]:
X_new = X_df.drop(columns=to_drop)

In [ ]:
print(np.mean(cross_val_score(clf, X_new, y, cv=skf)))

### Tipo filter: Feature selection de manera univariada

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
# Seleccionmos las 5 mejores usando chi2
feat_sel= SelectKBest(f_classif, k=5)
pip = Pipeline([('feat_sel', feat_sel), ('clf', clf)])

In [ ]:
print(np.mean(cross_val_score(pip, X, y, cv=skf)))

In [ ]:
res_k = []
range_k= np.arange(1, 25,1)

for k in range_k:
    feat_sel= SelectKBest(f_classif, k=k)
    pip = Pipeline([('feat_sel', feat_sel), ('clf', clf)])
#    print(np.mean(cross_val_score(pip, X, y, cv=skf)))
    res_k.append(np.mean(cross_val_score(pip, X, y, cv=skf)))
    
plt.plot(res_k)

Esto se podría haber hecho también usando `validation_curve`

In [ ]:
from sklearn.model_selection import validation_curve
train_scores, test_scores = validation_curve(pip, X, y, "feat_sel__k",  param_range=range_k, cv=skf)

In [ ]:
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.title("Validation Curve with Decision Tree")
plt.xlabel("$k$")
plt.ylabel("Score")
plt.ylim(0.85, 1.1)
lw = 2
plt.semilogx(range_k, train_scores_mean, label="Training score",
             color="darkorange", lw=lw)
plt.fill_between(range_k, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="darkorange", lw=lw)
plt.semilogx(range_k, test_scores_mean, label="Cross-validation score",
             color="navy", lw=lw)
plt.fill_between(range_k, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2,
                 color="navy", lw=lw)
plt.legend(loc="best")
plt.show()

¿Cómo accedemos a las mejores features? Con grid search se puede hacer. Al final, feature selection es como si ajustáramos un parámetro mas.

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'feat_sel__k': range_k}

grid = GridSearchCV(pip, param_grid,  cv=skf)

In [ ]:
grid.fit(X,y)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)
# Cuáles son estas features?
feats_selected = grid.best_estimator_.named_steps['feat_sel'].get_support()
print(feats_selected)
print(X_df.columns[feats_selected])

***Pregunta: ¿Por qué salen features correlacionadas?***

### Tipo wrapper: RECURSIVE FEATURE ELIMINATION

In [ ]:
from sklearn.feature_selection import RFECV

In [ ]:
rfecv = RFECV(estimator=clf, step=1, cv=skf,
              scoring='accuracy')
rfecv.fit(X, y)

In [ ]:
print(X_df.columns[rfecv.get_support()])
print(np.max(rfecv.grid_scores_))

In [ ]:
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score of number of selected features")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

### Tipo wrapper: Seleccionar de un modelo

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

In [ ]:
pip = Pipeline([('feat_sel', SelectFromModel(DecisionTreeClassifier())),
                ('clf', DecisionTreeClassifier())])
print(np.mean(cross_val_score(pip, X, y, cv = skf)))

Referencias: http://scikit-learn.org/stable/modules/feature_selection.html